In [1]:
# imports
import sys 
import random
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.colors as colors 
import matplotlib.cbook as cbook 
plt.rcParams.update({'font.size': 12}) 
import mpld3 
import ast
#import ROOT
import uproot3 as ur3
import uproot as ur4
import pandas as pd 
from tqdm import tqdm 
from scipy.optimize import curve_fit 
from scipy import stats
from scipy.signal import lfilter, filtfilt
from scipy.signal import savgol_filter
from scipy.signal import find_peaks
import scipy.integrate as integrate
from lmfit import Model
import threading
from pathlib import Path
from matplotlib import gridspec
import datetime
import matplotlib.ticker as mticker
import time
import pathlib
import awkward as ak
import pprint
import pickle
import struct
from os.path import exists

In [2]:
global Settings
Settings = {}

In [3]:
def GetBranchFromTree(filename,branchname):
    myFile = ur4.open(filename)
    
    nparray = np.array(myFile[Settings["General"]["Treename"]][branchname].array())
        
    myFile.close()
        
    return nparray

def checktype(value):
    if value.startswith('['):
        return ast.literal_eval(value)

    if value.isnumeric():
        return int(value)

    if '.' in value:
        splits = value.split('.',-1)
        if len(splits) == 2 and splits[0].isnumeric() and splits[1].isnumeric():
            return float(value)
        else:
            return value

    if '-' in value:
        splits = value.split('-',-1)
        if len(splits)>1 and splits[0]!='':
            return value
        else: 
            if '.' in value:
                return float(value)
            else:
                return int(value)
    return value

def LoadSettings(setfile):
    #Load Settings file
    print("Using "+setfile+" as Settings")
    with open(setfile, "r") as open_file:
        for line in open_file.readlines():
            if line.startswith('#') or len(line)<=1 or line.isspace():
                continue
            elif line.startswith('+'):  
                upper_key = line.strip("+").strip('\n')
            else:
                [lower_key,str_value] = line.split(" = ")
                value = checktype(str_value.strip('\n'))
                if upper_key not in Settings:
                    Settings.update({upper_key:{lower_key:value}})
                else:
                    Settings[upper_key].update({lower_key:value})
    open_file.close()

In [4]:
# First load settings
LoadSettings("./Settings")
pprint.pprint(Settings)

Using ./Settings as Settings


FileNotFoundError: [Errno 2] No such file or directory: './Settings'